Auteur: Michaël Leroy


 Ajout attribut tmja aux routes connues du fichier data gouv tmja

[+] Méthodologie

    - Interpolation linéaire des valeurs manquantes sur certain tronçons de routes ( traitement route par route)

    - Intersection entre les géométries (buffer) des edges et les tronçons  du fichier tmja
    - Vérifcation sur le nom de la route
    - affectation d'un attribut 'tmja' aux edges qui matchent


[-] Limitation:
    - Beaucoup de routes importantes ont une valeur nulle dans le datset...


In [ ]:
class CFG:
    area = [
        'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]
    area_name = 'france'      #'_'.join(area)

    

    bornes = False

    viz = False

    roads = (
        '|motorway'
        '|motorway_link'
        '|trunk'
        '|trunk_link'
        '|primary'
        '|primary_link'
        '|secondary'
        '|secondary_link'
        '|tertiary'
         '|tertiary_link'
    )

        # '|primary'
        # '|motorway_link'
        
        # '|primary_link'

        # '|secondary'
        # '|secondary_link'
        
        
        #  '|tertiary'
        #  '|tertiary_link'

    data_path = './osm_datas/'

import joblib
joblib.dump(CFG.area, 'CFG.pkl')    
import os
os.environ['USE_PYGEOS'] = '0'
%load_ext autoreload
%autoreload



%aimport OSM_helper_functions
%aimport OCM_helper_functions

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import osmnx as ox
import networkx as nx
from tqdm.notebook import tqdm
from OSM_helper_functions import *
from OCM_helper_functions import *


ox.settings.log_console=True
ox.settings.use_cache=True
''' To retrieve http data from OSMnx and cache them locally
    before building graph. Raise and exception if requests
    are all done. intercept then set to False and retry.'''
# ox.settings.cache_only_mode=True 

Administratives = {
    'Auvergne-Rhône-Alpes': ['Ain', 'Allier', 'Ardèche', 'Cantal', 'Drôme', 'Isère', 'Loire', 'Haute-Loire', 'Puy-de-Dôme', 'Rhône', 'Savoie', 'Haute-Savoie'],
    'Bourgogne-Franche-Comté': ['Côte-d\'Or', 'Doubs', 'Jura', 'Nièvre', 'Haute-Saône', 'Saône-et-Loire', 'Yonne', 'Territoire de Belfort'],
    'Brittany': ['Côtes-d\'Armor', 'Finistère', 'Ille-et-Vilaine', 'Morbihan'],
    'Centre-Val de Loire': ['Cher', 'Eure-et-Loir', 'Indre', 'Indre-et-Loire', 'Loir-et-Cher', 'Loiret'],
    # 'Corse': ['Corse-du-Sud', 'Haute-Corse'],
    'Grand Est': ['Ardennes', 'Aube', 'Marne', 'Haute-Marne', 'Meurthe-et-Moselle', 'Meuse', 'Moselle', 'Bas-Rhin', 'Haut-Rhin', 'Vosges'],
    'Hauts-de-France': ['Aisne', 'Nord', 'Oise', 'Pas-de-Calais', 'Somme'],
    'Île-de-France': ['Paris', 'Seine-et-Marne', 'Yvelines', 'Essonne', 'Hauts-de-Seine', 'Seine-Saint-Denis', 'Val-de-Marne', 'Val-d\'Oise'],
    'Normandie': ['Calvados', 'Eure', 'Manche', 'Orne', 'Seine-Maritime'],
    'Nouvelle-Aquitaine': ['Charente', 'Charente-Maritime', 'Corrèze', 'Creuse', 'Dordogne', 'Gironde', 'Landes', 'Lot-et-Garonne', 'Pyrénées-Atlantiques', 'Deux-Sèvres', 'Vienne', 'Haute-Vienne'],
    'Occitanie': ['Ariège', 'Aude', 'Aveyron', 'Gard', 'Haute-Garonne', 'Gers', 'Hérault', 'Lot', 'Lozère', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Tarn', 'Tarn-et-Garonne'],
    'Pays de la Loire': ['Loire-Atlantique', 'Maine-et-Loire', 'Mayenne', 'Sarthe', 'Vendée'],
    'Provence-Alpes-Côte d\'Azur': ['Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Bouches-du-Rhône', 'Var', 'Vaucluse']
}

Regions = [
    'Auvergne-Rhône-Alpes',
    'Bourgogne-Franche-Comté',
    'Brittany',
    'Centre-Val de Loire',
    # 'Corse',
    'Grand Est',
    'Hauts-de-France',
    'Île-de-France',
    'Normandie',
    'Nouvelle-Aquitaine',
    'Occitanie',
    'Pays de la Loire',
    'Provence-Alpes-Côte d\'Azur'
]

Name2num = {
    'Ain': '01',
    'Aisne': '02',
    'Allier': '03',
    'Alpes-de-Haute-Provence': '04',
    'Hautes-Alpes': '05',
    'Alpes-Maritimes': '06',
    'Ardèche': '07',
    'Ardennes': '08',
    'Ariège': '09',
    'Aube': '10',
    'Aude': '11',
    'Aveyron': '12',
    'Bouches-du-Rhône': '13',
    'Calvados': '14',
    'Cantal': '15',
    'Charente': '16',
    'Charente-Maritime': '17',
    'Cher': '18',
    'Corrèze': '19',
    'Corse-du-Sud': '2A',
    'Haute-Corse': '2B',
    'Côte-d\'Or': '21',
    'Côtes-d\'Armor': '22',
    'Creuse': '23',
    'Dordogne': '24',
    'Doubs': '25',
    'Drôme': '26',
    'Eure': '27',
    'Eure-et-Loir': '28',
    'Finistère': '29',
    'Gard': '30',
    'Haute-Garonne': '31',
    'Gers': '32',
    'Gironde': '33',
    'Hérault': '34',
    'Ille-et-Vilaine': '35',
    'Indre': '36',
    'Indre-et-Loire': '37',
    'Isère': '38',
    'Jura': '39',
    'Landes': '40',
    'Loir-et-Cher': '41',
    'Loire': '42',
    'Haute-Loire': '43',
    'Loire-Atlantique': '44',
    'Loiret': '45',
    'Lot': '46',
    'Lot-et-Garonne': '47',
    'Lozère': '48',
    'Maine-et-Loire': '49',
    'Manche': '50',
    'Marne': '51',
    'Haute-Marne': '52',
    'Mayenne': '53',
    'Meurthe-et-Moselle': '54',
    'Meuse': '55',
    'Morbihan': '56',
    'Moselle': '57',
    'Nièvre': '58',
    'Nord': '59',
    'Oise': '60',
    'Orne': '61',
    'Pas-de-Calais': '62',
    'Puy-de-Dôme': '63',
    'Pyrénées-Atlantiques': '64',
    'Hautes-Pyrénées': '65',
    'Pyrénées-Orientales': '66',
    'Bas-Rhin': '67',
    'Haut-Rhin': '68',
    'Rhône': '69',
    'Haute-Saône': '70',
    'Saône-et-Loire': '71',
    'Sarthe': '72',
    'Savoie': '73',
    'Haute-Savoie': '74',
    'Paris': '75',
    'Seine-Maritime': '76',
    'Seine-et-Marne': '77',
    'Yvelines': '78',
    'Deux-Sèvres': '79',
    'Somme': '80',
    'Tarn': '81',
    'Tarn-et-Garonne': '82',
    'Var': '83',
    'Vaucluse': '84',
    'Vendée': '85',
    'Vienne': '86',
    'Haute-Vienne': '87',
    'Vosges': '88',
    'Yonne': '89',
    'Territoire de Belfort': '90',
    'Essonne': '91',
    'Hauts-de-Seine': '92',
    'Seine-Saint-Denis': '93',
    'Val-de-Marne': '94',
    'Val-d\'Oise': '95'
}


AREA = []
for area in CFG.area:
    AREA.extend(Administratives[area] )
AREA

## Get Graph

In [ ]:
# file = os.path.join(CFG.data_path, f'[{CFG.area_name}]graph.graphml')
# try:
#     print(f'Load graph from disk for {CFG.area_name} roads,', end=' ')
#     G = ox.io.load_graphml(file)
#     print(f'{G.number_of_nodes()} nodes, {G.number_of_edges()} edges')

# except:
#     print('File not found')

In [ ]:
# G = ox.graph_from_gdfs(
#     joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gnodes.pkl')), 
#     joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'))
#     )


In [ ]:
edges = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges.pkl'))

In [ ]:
# edges

In [ ]:
# edges.crs

## View graph

In [ ]:
# Folium Map

# _, _ = graph2folium(G,viz=True)

In [ ]:
# Matplotlib Map

def visu(graph,title, folium=False):
    print(title)
    ox.plot_graph(G, node_size=3, node_color='red', edge_color='blue')
    
    print(graph.number_of_nodes(), graph.number_of_edges())
    if folium:
        return ox.folium.plot_graph_folium(graph, tiles='OpenStreetMap')

if CFG.viz:
    visu(G, 'Initial graph')    

# Get Nodes and Edges from graph

In [ ]:
# nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

# TJMA

## Get datas

In [ ]:
import geopandas as gpd

data_path = './datas'

# tmja = gpd.read_file('C:/Users/demo/Desktop/Lattitude/datas/tmja2018-shp/TMJA2018.shp')
file_name = 'TMJA2018'
ext = '.shp'
url = 'https://static.data.gouv.fr/resources/trafic-moyen-journalier-annuel-sur-le-reseau-routier-national/20200601-112539/tmja2018-shp.zip'
temp_path = 'temp_unzip'

try: 
    tmja = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    # From url
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    tmja = gpd.read_file(os.path.join(temp_path,file_name + ext))
    shutil.rmtree(temp_path)

    # From local file
    # tmja = gpd.read_file(os.path.join(data_path,file_name + ext))

    # avoiding multi dowloads for GES emmision lowering
    tmja.to_feather(os.path.join(data_path,file_name + '.feather'))

In [ ]:
import re

def format_string(input_string):
    # Remove leading digits
    formatted_string = re.sub(r'^\d+', '', input_string)
    
    # Replace zero numbers after letter with ' '
    formatted_string = re.sub(r'(?<=[A-Z])0+', ' ', formatted_string)
    
    # Remove leading zeros after a letter
    formatted_string = re.sub(r'(?<=[A-Z])0+(\d)', r'\1', formatted_string)
    
    # Keep only the first letter of the last group of letters
    formatted_string = re.sub(r'([A-Z])[A-Z]+$', r'\1', formatted_string)
    
    # Insert space between letter and number
    formatted_string = re.sub(r'(?<=[A-Z])(?=\d)', ' ', formatted_string)
    return formatted_string.strip()

# display(tmja.route.unique())
# display(tmja.route.apply(format_string).unique())

tmja['route_name'] = tmja.route.apply(format_string)

In [ ]:
tmja

In [ ]:
import numpy as np

print('missing before:', tmja.tmja.replace(0, np.nan).isna().sum() / tmja.shape[0])
for route in tmja.route:
    temp = tmja[tmja.route == route]\
            .sort_values('cumulD', ascending=True)
    # print(temp.shape)
    if temp.shape[0] > 2:
        interpolated = temp['tmja'].replace(0, np.nan)
        interpolated.interpolate(method='linear', limit_direction='both', inplace=True)
        # display([(x, y) for x,y in zip(temp.tmja, interpolated)])   
        # display(interpolated) 
        tmja.loc[interpolated.index,'tmja'] = interpolated.round(0)
        # break
    
print('missing after:', tmja.tmja.replace(0, np.nan).isna().sum() / tmja.shape[0])    

In [ ]:
tmja.fillna(0, inplace=True)

In [ ]:
tmja.tmja.isna().sum()

In [ ]:
# # TODO : for complete road with no data try to find tmja at start and end conections with other roads

# route_null = tmja.query('route_name == "N 44"').sort_values('cumulD', ascending=True)
# first_geometry = route_null.iloc[0]['geometry']
# start = first_geometry.coords[0]
# start_point = Point(start[1], start[0])

# tmja[start_point.buffer(4562100).intersects(tmja.geometry)]


# # Doesn't seem to work !!!!!

In [ ]:
from shapely.geometry import mapping

if CFG.viz:
    display(tmja.crs)


    tmja_84 = ox.projection.project_gdf(tmja, to_crs='epsg:4326', to_latlong=False)
    # tmja_84_area = tmja_84.query("depPrD == @dept")
    # temp_84 = tmja_84_area     #.head(1)


    # Graph folium map with temp_84 road in red
    m = ox.folium.plot_graph_folium(G, tiles='OpenStreetMap', popup_attribute='highway')
    folium.GeoJson(tmja_84['geometry'],
                    style_function=lambda x :{"fillColor": 'red',
                                                "fillOpacity": 1.0,
                                                "weight": 1,
                                                "color": 'red'},
                    name='tmja').add_to(m)
    display(m)
    del m, tmja_84


In [ ]:
# tmja.query("route_name == 'N 44'").sort_values('cumulD', ascending=True)#.tmja.plot(kind='line')

In [ ]:
# # Test if ref 'names' are equals to tmja.new_name
# def test(new_name, ref):
#     if type(ref) == list:
#         for n in ref:
#             if n == new_name:
#                 return True
#     elif ref == new_name:
#         return True
#     else:
#         return False        
    
# def test_attr(serie, text):
#     mask = []
#     for attr in serie:
#         if type(attr) == list:
#             # print(attr,'?',text)
#             submask = []
#             for t in attr:
#                 submask.append(t == text)
#             mask.append(any(submask))
#             # print(any(submask))
#         else:
#             mask.append(attr == text)   
#     return mask        

# Test if ref 'names' are equals to tmja.new_name
def  test_attr(serie, text):
    if type(text) == list:
        return serie.apply(lambda s : any([s == t.upper for t in text]))       
    else:
        return serie == text

In [ ]:
display(edges.head(1), tmja.head(1))

In [ ]:
# for n, (u, v, key, data) in enumerate(G.edges(keys=True, data=True)):
#     print(data["geometry"])
#     print(ox.projection.project_geometry(data["geometry"], crs=4326, to_crs=2154)[0])
#     break

In [ ]:

def tmja2graph(edges,tmja):
    fin = ' ' * 80
    edges['tmja'] = 0
    
    # Project the graph to lambert
    # print('Project graph to Lambert...', end=' ')
    # graph = ox.projection.project_graph(graph, to_crs=2154)
    _edges = edges.copy()
    print('edges projection to Lambert')
    _edges = ox.projection.project_gdf(_edges, to_crs=2154)
    print('edges buffering')
    _edges['geometry'] = _edges['geometry'].buffer(70, cap_style='flat')
    # print(' done.')
    count = 0
    no_match = []
    total = tmja[tmja['tmja'] > 0].shape[0]
    print('working')
    for n, row in tmja[tmja['tmja'] > 0].iterrows():
        print(f'{n + 1}/{total} ~ {(n + 1) / total:.2%} : ', end=' ')# end=' '*50 + '\r')
        
        # edges += 1
        # project geometry to Lambert
        # edge_geometry = ox.projection.project_geometry(data["geometry"], crs=4326, to_crs=2154)[0]
        # display( edge_geometry)
        # tmja_geometry_expanded = tmja['geometry'].buffer(50, cap_style='flat')
        # display( edge_geometry_expanded)

        matching_edges = _edges[ _edges['geometry'].intersects(row['geometry']) ]
        
        if not matching_edges.empty:
            print('matching _edges: ', matching_edges.shape[0], end= ' ')
            matching_edges = matching_edges[test_attr(matching_edges['ref'],row.route_name)]
            if not matching_edges.empty:
                attribute_value = row["tmja"]
                print(f'retained: {matching_edges.shape[0]}', end=fin + '\r')
                for idx in matching_edges.index:
                    edges.loc[idx,'tmja'] = attribute_value
                    count +=1
        else:
            no_match.append(n)
            print('no matching edges', end=fin * 2 + '\r')    
    no_match = tmja[tmja['tmja'] > 0].loc[no_match].index
    del _edges, matching_edges
    print('Number of egdes with tmja added ',count,' on ', edges.shape[0])    
    # return the projected graph to GPS 84    
    # print('Reproject and return graph.')
    # return ox.projection.project_graph(graph, to_crs=4326)

    edges['tmja'] = edges['tmja'].apply(lambda x: np.nan if x == 0 else x)
    return edges, no_match


edges, no_match = tmja2graph(edges, tmja)

display(tmja.loc[no_match])

if CFG.viz:
    graphplot(G,'tmja' , title=CFG.area_name )

# Save edges

In [ ]:

joblib.dump(edges, os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_with_known_tmja_mthd2.pkl'),compress=3)

# Compare method 1 & 2

In [ ]:
edges_1 = joblib.load( os.path.join(CFG.data_path, f'[{CFG.area_name}]Gedges_with_known_tmja_mthd1.pkl'))

In [ ]:
diff = edges.reset_index().sort_values(by=['u','v','key']).reset_index()['tmja']\
        .compare(edges_1.reset_index().sort_values(by=['u','v','key']).reset_index()['tmja'])
diff.info()

In [ ]:
diff

-----------------------------

In [ ]:
******* Fin du taritement des données **************

# Betweeness

In [ ]:
def split_dict(dct):
    dct_stations={}
    dct_nodes = {}
    for key, value in dct.items():
        if str(key) == key:
            dct_stations[key] =  value
        else:
            dct_nodes[key] = value    
    display(f'Stations: {len(dct_stations)} Nodes: { len(dct_nodes)}')
    return dct_stations, dct_nodes
    

In [ ]:
closeness = nx.closeness_centrality(
                G2, 
                u=None, 
                distance='distance', 
                wf_improved=True
                )
clsns_stations, clsns_nodes = split_dict(closeness)

In [ ]:
betweeness = nx.betweenness_centrality(
    G2, 
    k=None, 
    normalized=True, 
    weight='distance', 
    endpoints=False, 
    seed=42)

btwns_stations, btwns_nodes = split_dict(betweeness)

## Add scores to nodes

In [ ]:
for n in G2.nodes():
    G2.nodes[n]['closeness'] = closeness[n]
    G2.nodes[n]['betweeness'] = betweeness[n]

# Save Scores

In [ ]:
# Define the file namer the graph file
data_path = './datas'
area_name = [s.replace("\'","@") for s in selection]
filename = f"{area_name}_drive_charge_tmja_scores.graphml"

map_save_graph(G2, filename)


-------------------------------------------------

In [ ]:
if CFG.bornes:
    stations_df =pd.concat(
        [
        pd.DataFrame.from_dict(clsns_stations, orient='index').rename(columns={0:'closeness'}),
        pd.DataFrame.from_dict(btwns_stations, orient='index').rename(columns={0:'betweeness'})    
        ], axis=1
    )
    stations_df.describe()

In [ ]:
nodes_df =pd.concat(
    [
    pd.DataFrame.from_dict(clsns_nodes, orient='index').rename(columns={0:'closeness'}),
    pd.DataFrame.from_dict(btwns_nodes, orient='index').rename(columns={0:'betweeness'})    
    ], axis=1
)
nodes_df.describe()

--------------------------------------------

In [ ]:
from matplotlib import pyplot  as plt

if CFG.bornes:
    stations_df.plot(kind='hist',bins=100,  alpha=.5)
    plt.gcf().suptitle('Stations')

In [ ]:
if CFG.bornes:
    nodes_df.plot(kind='hist',bins=100,  alpha=.5)
    plt.gcf().suptitle('Nodes')